# Generalizations of Max-Flow


## Min-Cost Flow

Flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

Data was picked from some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/mp-testdata/mincost/gte/info) file. 

In [11]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    
    f = open(infile)
    G = nx.DiGraph()
    count = 0
    
    #Function
    for line in f.readlines():
        count = count+1
        
        if line != '\n':
            start = line.split()[0]     
            
            #checking if the line starts with p, n or a and using the description in the info file
            if start == 'p':
                nodes = int(line.split()[2])
                edges = int(line.split()[3])
            
            if start == 'n':
                line_split = line.split()
                node, demand = line_split[1:3]
                G.add_node(node)
                G.node[node]['demand'] = float(demand)
            
            if start == 'a':
                line_split = line.split()
                e_tail, e_head, e_cap_lower, e_cap, e_wt = line_split[1:6]
                
                if e_cap < 0:
                    e_cap = float('inf')
                else: 
                    e_cap = float(e_cap)
                    
                #For the first edge between two nodes
                if (e_tail, e_head) not in G.edges(): 
                    G.add_edge(e_tail, e_head)
                    G.edge[e_tail][e_head]['capacity'] = float(e_cap)
                    G.edge[e_tail][e_head]['weight'] = float(e_wt)   
                 
                #If multiple edges are found, add a new node   
                else:                                      
                    e_int = e_tail + '_' + e_head + '_' + 'int' + '_' + str(count)
                    G.add_edge(e_tail, e_int)
                    G.edge[e_tail][e_int]['capacity'] = float(e_cap)
                    G.edge[e_tail][e_int]['weight'] = float(e_wt)/2.0  
                    
                    G.add_edge(e_int, e_head)
                    G.edge[e_int][e_head]['capacity'] = float(e_cap)
                    G.edge[e_int][e_head]['weight'] = float(e_wt)/2.0
    for node in G.node:
        if 'demand' not in G.node[node]:
            G.node[node]['demand'] = 0
        
    return G
    pass

In [ ]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

## Linear Programming

Function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).


In [13]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
   
    # dictionary of weights 
    weights = {}
    for (i,j) in G.edges():
        weights[(i,j)] = G.edge[i][j]['weight']
    
    # dictionary of capacities
    caps = {}
    for (i,j) in G.edges():
        caps[(i,j)] = G.edge[i][j]['capacity']

    # dictionary of demands
    demands = {}
    for n in G.nodes():
        demands[n] = G.node[n]['demand']
        
    # dictionary of flow variables for L.P.
    flow = pulp.LpVariable.dicts("fl",G.edges(),0,cat='Integer')

    # set upper bound of flow variable to capacity of edge
    for i in flow.viewkeys():
        flow[i].bounds(0,caps[i])

    # create L.P. problem
    lp = pulp.LpProblem("Min-Cost Flow Problem",pulp.LpMinimize)

    # objective function
    lp += pulp.lpSum([flow[i]*weights[i] for i in G.edges()]), "Total Cost of Flow"

    # constraints
    N = G.nodes()
    for n in N:
        lp += (pulp.lpSum([flow[(j,i)] for (j,i) in G.edges() if i==n]) - pulp.lpSum([flow[(i,j)] for (i,j) in G.edges() if i==n])) == demands[n]
    
    # solving problem
    lp.solve()
    
    # returning min-cost flow
    return pulp.value(lp.objective)
    
    pass